# Lekce 4

Datové soubory se nacházejí zde:

- [ioef.csv](ioef.csv),
- [vysledky.csv](vysledky.csv).

## Metoda rank

Metoda `rank` "očísluje" řádky podle zvoleného sloupce. To se hodí třeba v případě, že chceme zjistit a porovnat pořadí záznamů podle nějakého kritéria. Níže máme načtený dataset Indexu ekonomické svobody
(Index of Economic Freedom) z [oficiálního webu](https://www.heritage.org/index/). Index na základě vybraných kritérií vyhodnocuje, nakolik je země ekonomicky svobodná.

In [31]:
import pandas
# načtení souboru
ioef = pandas.read_csv("ioef.csv")
ioef.head()

Id         Name   Short Name ISO Code  Index Year  Overall Score  \
0   1  Afghanistan  Afghanistan       AF        2023            NaN   
1   2      Albania      Albania       AL        2023           65.3   
2   3      Algeria      Algeria       DZ        2023           43.2   
3   4       Angola       Angola       AO        2023           53.0   
4   5    Argentina    Argentina       AR        2023           51.0   

   Property Rights  Judicial Effectiveness  Government Integrity  Tax Burden  \
0              5.8                     5.4                  12.7         NaN   
1             55.9                    49.2                  35.5        89.1   
2             27.8                    29.5                  28.4        71.9   
3             41.1                    24.8                  22.9        86.5   
4             34.6                    56.8                  42.8        66.9   

   Government Spending  Fiscal Health  Business Freedom  Labor Freedom  \
0                  NaN            NaN              34.6           45.1   
1                 71.0           58.2              70.7           49.7   
2                 50.7           12.1              53.6           51.4   
3                 86.9           85.1              41.6           53.9   
4                 53.9           33.6              55.5           55.1   

   Monetary Freedom  Trade Freedom  Investment Freedom  Financial Freedom  
0               NaN            NaN                 NaN                NaN  
1              81.3           82.8                70.0               70.0  
2              75.1           57.5                30.0               30.0  
3              58.9           64.8                30.0               40.0  
4              36.5           61.2                55.0               60.0

Data zobrazují hodnoty od roku 1995 do roku 2021. Začněme tím, že si určíme pořadí země podle celkvého indexu (sloupec `Overall Score`). K tomu využijeme metodu `rank`. Protože ale potřebujeme určit pořadí vždy pro konkrétní rok, přidáme metodu `groupby` podle sloupce `Index Year`. Tím zajistíme, že číslování pro každý rok začne od 1.

Musíme též nastavit, aby bylo pořadí udělování sestupně, protože čím vyšší je dosažené skóre, tím výše se země na žebříčku ekonomických svobod nachází. To zařídí parametr `ascending`.

Kromě nastavení sloupce má metoda `rank` důležitý parametr `method`. Ta vyřeší případ, že skupina řádků má stejnou hodnotu. Využijeme parametr `dense`, který celé skupině přiřadí hodnotu o 1 vyšší než předchozí skupině. Pro kontrolu si do zvláštní tabulky dáme hodnoty z roku 2023.

In [40]:
# přidání sloupečku rank (zpočítá pořadí) - pořadí dle sloupečku Overall Score
# ioef["Rank"] = ioef["Overall Score"].rank()
# chceme-li zpět jen některé sloupečky:
ioef = ioef[["Name", "Index Year", "Overall Score"]]
# přidání sloupečku ranku (pořadí) ale jen v rámci jednoho roku - groupby(["Index Year"])

ioef["Rank"] = ioef.groupby(["Index Year"])["Overall Score"].rank(method="min", ascending=False)
# parametr ascending=False-tzn. 1 bude mít stát který je nejlepší
# parametr method="min" všem státům dá to nejnižší, celočíselné pořadí (menší)
ioef.head()


# ioef_2023 = ioef[ioef["Index Year"] == 2023]
# ioef_2023.head()


Name  Index Year  Overall Score  Rank
5226       Hong Kong        1995           88.6   1.0
5302       Singapore        1995           86.3   2.0
5329  United Kingdom        1995           77.9   3.0
5330   United States        1995           76.7   4.0
5166         Bahrain        1995           76.2   5.0

In [41]:
# seřadíme řádky pomocí metody sort_values()
# chceme seřadit dle roku a pořadí - od nejstrašího roku a od 1 pořadí
# ioef = ioef.sort_values(["Index Year", "Rank"])
ioef.head()



Name  Index Year  Overall Score  Rank
5226       Hong Kong        1995           88.6   1.0
5302       Singapore        1995           86.3   2.0
5329  United Kingdom        1995           77.9   3.0
5330   United States        1995           76.7   4.0
5166         Bahrain        1995           76.2   5.0

Podíváme se nyní na pět států s nejnižším pořadím.

In [ ]:
ioef_2023.sort_values(["Rank"]).head()

In [ ]:
# Nyní se podíváme jak se vyvíjelo pořadí ČR, využijeme metodu shift(), abychom do každého řádku přidali pořadí
# v předcházejícím roce
ioef_sorted = ioef.sort_values(["Name", "Index Year"])
ioef_sorted["Rank Previous Year"] = ioef_sorted.groupby(["Name"])["Rank"].shift()
ioef_sorted.head()

In [42]:
# chceme-li dat jen o jednom konkrétním státě např. ČR
ioef_cz = ioef[ioef["Name"] == "Czech Republic"]
ioef_cz.head()

Name  Index Year  Overall Score  Rank
5199  Czech Republic        1995           67.8  23.0
5016  Czech Republic        1996           68.1  28.0
4833  Czech Republic        1997           68.8  24.0
4650  Czech Republic        1998           68.4  28.0
4467  Czech Republic        1999           69.7  22.0

In [49]:
# pomocí metody shift chceme porovnat akutální pořadí s předchozím pořadím do jednoho řádku
ioef_cz["Rank Previous Year"] = ioef_cz["Rank"].shift(-1)
# chceme-li porovna předchozí rok s letošním - výsledek je v novém sloupci - rozdíl
ioef_cz["Rank Change"] = ioef_cz["Rank Previous Year"] - ioef_cz["Rank"]
ioef_cz.head()


C:\Users\lucie\AppData\Local\Temp\ipykernel_26556\3521880057.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ioef_cz["Rank Previous Year"] = ioef_cz["Rank"].shift(-1)
C:\Users\lucie\AppData\Local\Temp\ipykernel_26556\3521880057.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ioef_cz["Rank Change"] = ioef_cz["Rank Previous Year"] - ioef_cz["Rank"]


Name  Index Year  Overall Score  Rank  Rank Previous Year  \
5199  Czech Republic        1995           67.8  28.0                24.0   
5016  Czech Republic        1996           68.1  24.0                28.0   
4833  Czech Republic        1997           68.8  28.0                22.0   
4650  Czech Republic        1998           68.4  22.0                30.0   
4467  Czech Republic        1999           69.7  30.0                26.0   

      Rank Change  
5199         -4.0  
5016          4.0  
4833         -6.0  
4650          8.0  
4467         -4.0

In [3]:
ioef_2023.sort_values(["Rank"]).head()

Id         Name   Short Name ISO Code  Index Year  Overall Score   
144  147    Singapore    Singapore       SG        2023           83.9  \
155  158  Switzerland  Switzerland       CH        2023           83.8   
76    78      Ireland      Ireland       IE        2023           82.0   
157  160       Taiwan       Taiwan       TW        2023           80.7   
117  120  New Zealand   NewZealand       NZ        2023           78.9   

     Property Rights  Judicial Effectiveness  Government Integrity   
144             94.0                    58.3                  91.2  \
155             94.2                    97.8                  92.3   
76              92.9                    93.9                  82.8   
157             81.9                    94.7                  76.3   
117             87.8                    94.7                  96.8   

     Tax Burden  Government Spending  Fiscal Health  Business Freedom   
144        90.6                 89.0           78.0              86.9  \
155        70.6                 63.9           95.4              84.3   
76         78.4                 80.5           86.8              87.2   
157        79.3                 91.1           93.6              84.3   
117        66.6                 49.7           72.2              88.8   

     Labor Freedom  Monetary Freedom  Trade Freedom  Investment Freedom   
144           77.3              81.9           95.0                85.0  \
155           60.5              85.1           86.6                85.0   
76            61.2              81.5           78.6                90.0   
157           69.1              82.5           85.8                70.0   
117           71.5              78.7           90.4                70.0   

     Financial Freedom  Rank  
144               80.0   1.0  
155               90.0   2.0  
76                70.0   3.0  
157               60.0   4.0  
117               80.0   5.0

Nyní porovnáme, jak se vyvíjelo pořadí České republiky. K tomu využijeme metodu `shift()`, abychom do každého řádku přidali pořadí v předcházejícím roce.

In [54]:
# nejpreve seřadíme řádky dle jména a roku: 
ioef_sorted = ioef.sort_values(["Name", "Index Year"], ascending=[True, False])
# parametr ascending=[True, False] - říká, ať se státy řadí dle abecedy vzestupně, ale dle roku sestupně

# shift prováději jen v rámci sloupečku Name - pokud je jiný stát, tak tam nedávej žádnou hodnotu-pomocí metody groupby("Name")
ioef_sorted["Rank Previous Year"] = ioef_sorted.groupby("Name")["Rank"].shift(-1)
ioef_sorted["Rank Change"] = ioef_sorted["Rank Previous Year"] - ioef_sorted["Rank"]

ioef_sorted.head()

Name  Index Year  Overall Score   Rank  Rank Previous Year  \
0    Afghanistan        2023            NaN    NaN                 NaN   
184  Afghanistan        2022            NaN    NaN               145.0   
370  Afghanistan        2021           53.0  145.0               136.0   
556  Afghanistan        2020           54.7  136.0               152.0   
742  Afghanistan        2019           51.5  152.0               154.0   

     Rank Change  
0            NaN  
184          NaN  
370         -9.0  
556         16.0  
742          2.0

Vidíme například, že k největšímu zlepšení došlo v roce 2023 (o 6 příček)

In [5]:
ioef_sorted["Rank Change"] = ioef_sorted["Rank"] - ioef_sorted["Rank Previous Year"]
ioef_sorted_czech = ioef_sorted[ioef_sorted["Name"] == "Czech Republic"]
ioef_sorted_czech.tail()

Id            Name     Short Name ISO Code  Index Year  Overall Score   
785  44  Czech Republic  CzechRepublic       CZ        2019           73.7  \
599  44  Czech Republic  CzechRepublic       CZ        2020           74.8   
413  44  Czech Republic  CzechRepublic       CZ        2021           73.8   
227  44  Czech Republic  CzechRepublic       CZ        2022           74.4   
43   44  Czech Republic  CzechRepublic       CZ        2023           71.9   

     Property Rights  Judicial Effectiveness  Government Integrity   
785             74.8                    47.6                  52.1  \
599             76.8                    49.9                  64.2   
413             76.2                    56.8                  64.4   
227             88.8                    81.8                  59.6   
43              88.5                    81.9                  60.3   

     Tax Burden  ...  Fiscal Health  Business Freedom  Labor Freedom   
785        82.6  ...           97.6              72.4           78.1  \
599        82.0  ...           97.8              69.7           77.6   
413        79.1  ...           98.1              68.8           77.1   
227        78.9  ...           93.2              80.6           56.5   
43         79.3  ...           73.5              76.9           56.1   

     Monetary Freedom  Trade Freedom  Investment Freedom  Financial Freedom   
785              81.5           86.0                80.0               80.0  \
599              80.8           86.4                80.0               80.0   
413              79.7           84.0                70.0               80.0   
227              79.0           79.2                70.0               80.0   
43               78.0           78.6                70.0               80.0   

     Rank  Rank Previous Year  Rank Change  
785  23.0                24.0         -1.0  
599  23.0                23.0          0.0  
413  26.0                23.0          3.0  
227  20.0                26.0         -6.0  
43   21.0                20.0          1.0  

[5 rows x 21 columns]

## Metoda `apply`

Metoda `apply()` nám umožňuje používat vlastní funkce. Pokud tedy nějakou úpravu dat sám modul `pandas` neumí, případně pokud by bylo jejich použití příliš složité a nepřehledné, můžeme si vytvořit vlastní funkci.

Vlastní funkci vytváříme pomocí klíčového slova `def`. Poté přidáme název funkce a parametry, které příjmá. U `pandas` máme možnost použít funkci dvěma způsoby:

- pracovat s hodnotou v konkrétním sloupci,
- pracovat se všemi hodnotami v řádku.

Vyzkoušíme si použití pro celou tabulku. V tabulce [vysledky.csv](vysledky.csv) máme známky z maturitní zkoušky a naším úkolem je zjistit, zda člověk prospěl s vyznamenáním, prospěl (bez vyznamenání) či neprospěl. Pravidla jsou následující:

- pokud má člověk průměr do 1.5 a nemá horší známku než 2, prospěl s vyznamenáním,
- pokud dostal známku 5, pak neprospěl,
- ve všech ostatních případech prospěl.

Vytvoříme si funkci `evaluate_result()`, která bude mít jako jeden parametr `row` (řádek tabulky). Parametr `row()` je série, tj. můžeme vybírat konkrétní hodnoty (např. známku z vybraného předmětu), dále můžeme využívat agregační funkce jako průměr či maximum.

In [55]:
import pandas

def evaluate_result(row):
    row = row.iloc[2:]
    if row.mean() <= 1.5 and row.max() <= 2:
        return "Prospěl(a) s vyznamenáním"
    elif row.max() == 5:
        return "Neprospěl(a)"
    else:
        return "Prospěl(a)"

data = pandas.read_csv("vysledky.csv")
data["vysledek"] = data.apply(evaluate_result, axis=1)
data.head()

Jméno  Poplatek  Český jazyk  Anglický jazyk  Informatika  \
0    Mirek Dušín  1.4.2023            2               2            1   
1  Jarka Metelka  3.4.2023            3               5            3   
2   Jindra Hojer  7.4.2023            2               2            1   
3     Červenáček       NaN            1               1            1   
4    Rychlonožka       NaN            4               3            2   

   Matematika                   vysledek  
0           1  Prospěl(a) s vyznamenáním  
1           1               Neprospěl(a)  
2           3                 Prospěl(a)  
3           4                 Prospěl(a)  
4           4                 Prospěl(a)

Nyní si vyzkoušíme, jak funguje výběr jedné hodnoty nebo více hodnot. Uvažujme například, že nyní řešíme přijetí studentů/studentek na vysokou školu s matematickým zaměřením. Jako první zkontrolujeme, zda student zaplatil administrativní poplatek za podání přihlášky. Ve sloupci `Poplatek` vidíme datum (pokud je poplatek zaplacen) nebo prázdnou hodnotu (pokud zaplacen není). Na začátku tedy využijeme funkce `pandas.isnull()`, která vrátí `True`, pokud je pro daný řádek daný sloupec prázdný, a `False`, pokud pro daný řádek ve sloupci nějaká hodnota je. Dále řešíme, zda má být student přijet bez přijímací zkoušky. Škola umožňuje přijetí studentů bez přijímací zkoušky, pokud mají z maturity jedničku z matematiky a současně mají průměr všech známek menší nebo roven 2.

In [7]:
def evaluate_application(row):
    row = row.iloc[1:]
    if pandas.isnull(row["Poplatek"]):
        return "Vyřazen - nezaplatil"
    elif row["Matematika"] == 1 and row["Český jazyk":"Matematika"].mean() <= 2:
        return "Přijat bez PZ"
    else:
        return "Musí absolvovat PZ"
    
data["prijimaci_zkouska"] = data.apply(evaluate_application, axis=1)
data.head()

Jméno  Poplatek  Český jazyk  Anglický jazyk  Informatika   
0    Mirek Dušín  1.4.2023            2               2            1  \
1  Jarka Metelka  3.4.2023            3               5            3   
2   Jindra Hojer  7.4.2023            2               2            1   
3     Červenáček       NaN            1               1            1   
4    Rychlonožka       NaN            4               3            2   

   Matematika                   vysledek     prijimaci_zkouska  
0           1  Prospěl(a) s vyznamenáním         Přijat bez PZ  
1           1               Neprospěl(a)    Musí absolvovat PZ  
2           3                 Prospěl(a)    Musí absolvovat PZ  
3           4                 Prospěl(a)  Vyřazen - nezaplatil  
4           4                 Prospěl(a)  Vyřazen - nezaplatil

## Čtení na doma - volání s parametry

Metoda `apply()` umožňuje přidání vlastních hodnot při volání funkce. Pokud bychom například chtěli změnit hranici pro přijetí studenta (studentky) mezi roky, namísto úpravy vnitřku funkce můžeme přidat parametr `hranice_prumer`. Jeho konkrétní hodnotu pak zadáme jako parametr `args`. Protože obecně můžeme mít parametrů více, zapisujeme jejich hodnoty do seznamu.

In [8]:
def evaluate_application(row, hranice_prumer):
    row = row.iloc[1:]
    if pandas.isnull(row["Poplatek"]):
        return "Vyřazen - nezaplatil"
    elif row["Matematika"] == 1 and row["Český jazyk":"Matematika"].mean() <= hranice_prumer:
        return "Přijat bez PZ"
    else:
        return "Pozvat na PZ"
    
data["prijimaci_zkouska"] = data.apply(evaluate_application, axis=1, args=[1.5])
data.head()

Jméno  Poplatek  Český jazyk  Anglický jazyk  Informatika   
0    Mirek Dušín  1.4.2023            2               2            1  \
1  Jarka Metelka  3.4.2023            3               5            3   
2   Jindra Hojer  7.4.2023            2               2            1   
3     Červenáček       NaN            1               1            1   
4    Rychlonožka       NaN            4               3            2   

   Matematika                   vysledek     prijimaci_zkouska  
0           1  Prospěl(a) s vyznamenáním         Přijat bez PZ  
1           1               Neprospěl(a)          Pozvat na PZ  
2           3                 Prospěl(a)          Pozvat na PZ  
3           4                 Prospěl(a)  Vyřazen - nezaplatil  
4           4                 Prospěl(a)  Vyřazen - nezaplatil

# Cvičení

## Přijímací zkouška

Uvažujme nyní vyhodnocení přijímací zkoušky na střední školu technického zaměření, které jsou uloženy v souboru [prijimaci_zkousky.csv](prijimaci_zkousky.csv). Uchazeči a uchazečky musejí absolvovat tři písemné testy: test z anglického jazyka, matematiky a českého jazyka. Pokud dostane z některého z testů méně než 60 bodů, dostane hodnocení 0. Pokud má ze všech testů více než 60 bodů, je jeho skóre rovno součtu bodů ze všech tří testů. Dále uvažujeme, že za aktivity uchazeče přičítáme bonusové body:

- za 1. až 10. místo v některé z olympiád 10 bonusových bodů,
- za účast na letní škole dostane 5 bonusových bodů.

Vytvoř funkci `get_points()`, která přidělí každému z uchazečů počet bodů na základě daných kritérií. Dále vytvoř novou tabulku, která obsahuje pouze uchazeče a uchazečky, kteří mají nenulové hodnocení. Dále využij metodu `rank()` k výpočtu pořadí uchazeče/uchazečky v rámci daného oboru, a data seřaď podle oboru a počtu bodů. Nakonec vytvoř tabulku s přijatými uchazeči, kde budou uchazeči hodnocení od 1. do 30. místa pro daný obor.


Níže jsou data. Informace o olympiádách jsou ve sloupci `souteze`. Jsou tam uvedeny pouze údaje, které se započítávají do zkoušky, tj. získání 1. až 10. místa.

In [9]:
import pandas

data = pandas.read_csv("prijimaci_zkousky.csv")
data.head()

prijmeni krestni_jmeno              obor  body_aj  body_mat  body_cj   
0  Květoslava     Růžičková   Elektrotechnika       62        48       51  \
1         Jan       Moravec  Technické lyceum       81        74       66   
2       Denis         Fiala       Informatika       63        69       65   
3       Luboš         Šimek       Informatika       78        79       65   
4       Vilém       Novotný       Informatika       84        68       78   

  letni_skola                           souteze  
0          ne                               NaN  
1          ne  7. místo (matematická olympiáda)  
2          ne                               NaN  
3          ne                               NaN  
4         ano                               NaN

### Nápověda

Níže jsou data včetně výpočtu bodů.

In [10]:
data = pandas.read_csv("prijimaci_zkouska_body.csv")
data.head(15)

Unnamed: 0    prijmeni krestni_jmeno              obor  body_aj  body_mat   
0            0  Květoslava     Růžičková   Elektrotechnika       62        48  \
1            1         Jan       Moravec  Technické lyceum       81        74   
2            2       Denis         Fiala       Informatika       63        69   
3            3       Luboš         Šimek       Informatika       78        79   
4            4       Vilém       Novotný       Informatika       84        68   
5            5   Drahomíra    Štěpánková   Elektrotechnika       69        74   
6            6      Vlasta      Horáková  Technické lyceum       53        61   
7            7       Šimon      Pospíšil  Technické lyceum       70        66   
8            8    Jaroslav          Říha   Elektrotechnika       79        71   
9            9      Renata          Malá   Elektrotechnika       71        66   
10          10       Vilém        Dvořák   Elektrotechnika       98        92   
11          11       Radka       Dušková  Technické lyceum       88        70   
12          12       Luděk    Kratochvíl   Elektrotechnika       92        89   
13          13      Kamila       Vacková  Technické lyceum       56        41   
14          14      Simona       Němcová  Technické lyceum       63        56   

    body_cj letni_skola                           souteze  body  
0        51          ne                               NaN     0  
1        66          ne  7. místo (matematická olympiáda)   231  
2        65          ne                               NaN   197  
3        65          ne                               NaN   222  
4        78         ano                               NaN   235  
5        60          ne                               NaN   203  
6        57          ne                               NaN     0  
7        68          ne                               NaN   204  
8        63          ne                               NaN   213  
9        75         ano                               NaN   217  
10       80          ne                               NaN   270  
11       66          ne                               NaN   224  
12       93          ne                               NaN   274  
13       57          ne                               NaN     0  
14       61          ne                               NaN     0

Níže jsou data včetně pořadí (po vyřazení uchazečů, kteří měli jeden z testů na méně než 60 bodů).

In [11]:
data = pandas.read_csv("prijimaci_zkouska_poradi.csv")
data.head(15)

Unnamed: 0  index  prijmeni krestni_jmeno             obor  body_aj   
0           10     12     Luděk    Kratochvíl  Elektrotechnika       92  \
1           89    143   Daniela      Poláková  Elektrotechnika      102   
2            8     10     Vilém        Dvořák  Elektrotechnika       98   
3           98    160   Barbora       Vacková  Elektrotechnika       90   
4           45     72   Lubomír         Holub  Elektrotechnika       96   
5           85    138    Anežka      Kolářová  Elektrotechnika       92   
6          103    166     Radek         Bláha  Elektrotechnika      103   
7           78    123  Karolína      Nováková  Elektrotechnika       84   
8          115    186     Vilém         Marek  Elektrotechnika       92   
9           74    118      Erik         Beneš  Elektrotechnika       73   
10          36     57      René      Pospíšil  Elektrotechnika       88   
11          29     42   Vojtěch       Doležal  Elektrotechnika       83   
12          33     48     Marie        Říhová  Elektrotechnika       91   
13         100    162    Matyáš         Marek  Elektrotechnika       79   
14          90    144     Marie      Horáková  Elektrotechnika       75   

    body_mat  body_cj letni_skola                            souteze  body   
0         89       93          ne                                NaN   274  \
1         78       94          ne                                NaN   274   
2         92       80          ne                                NaN   270   
3         83       87          ne  2. místo (přírodopisná olympiáda)   270   
4         90       82          ne                                NaN   268   
5         78       91          ne                                NaN   261   
6         73       83          ne                                NaN   259   
7         95       79          ne                                NaN   258   
8         85       79          ne                                NaN   256   
9         87       86         ano                                NaN   251   
10        67       90         ano                                NaN   250   
11        80       86          ne                                NaN   249   
12        71       78         ano                                NaN   245   
13        77       83         ano                                NaN   244   
14        84       84          ne                                NaN   243   

    poradi  
0      1.0  
1      1.0  
2      3.0  
3      3.0  
4      5.0  
5      6.0  
6      7.0  
7      8.0  
8      9.0  
9     10.0  
10    11.0  
11    12.0  
12    13.0  
13    14.0  
14    15.0

Níže jsou data včetně rozhodnutí o přijetí.

In [12]:
data = pandas.read_csv("prijimaci_zkouska_rozhodnuti.csv")
data.head()

Unnamed: 0  index prijmeni krestni_jmeno             obor  body_aj   
0          10     12    Luděk    Kratochvíl  Elektrotechnika       92  \
1          89    143  Daniela      Poláková  Elektrotechnika      102   
2           8     10    Vilém        Dvořák  Elektrotechnika       98   
3          98    160  Barbora       Vacková  Elektrotechnika       90   
4          45     72  Lubomír         Holub  Elektrotechnika       96   

   body_mat  body_cj letni_skola                            souteze  body   
0        89       93          ne                                NaN   274  \
1        78       94          ne                                NaN   274   
2        92       80          ne                                NaN   270   
3        83       87          ne  2. místo (přírodopisná olympiáda)   270   
4        90       82          ne                                NaN   268   

   poradi prijat  
0     1.0    Ano  
1     1.0    Ano  
2     3.0    Ano  
3     3.0    Ano  
4     5.0    Ano

In [13]:
data = pandas.read_csv("prijimaci_zkouska_rozhodnuti.csv")
data.tail()

Unnamed: 0  index  prijmeni  krestni_jmeno              obor  body_aj   
117          54     87  Veronika       Poláková  Technické lyceum       68  \
118          25     38    Monika  Kratochvílová  Technické lyceum       60   
119          11     15      Erik         Dvořák  Technické lyceum       68   
120         119    195   Naděžda      Čermáková  Technické lyceum       71   
121          21     31       Jan         Kadlec  Technické lyceum       61   

     body_mat  body_cj letni_skola souteze  body  poradi prijat  
117        68       61         ano     NaN   202    27.0    Ano  
118        76       65          ne     NaN   201    28.0    Ano  
119        71       61          ne     NaN   200    29.0    Ano  
120        61       67          ne     NaN   199    30.0    Ano  
121        67       62          ne     NaN   190    31.0     Ne

### Bonusy

- Zájemcům o studium na oborech Elektrotechnika a Informatika přiděluj bonusové body pouze za fyzikální nebo matematickou olympiádu, zájemcům o lyceum i za ostatní olympiády.
- Za 1. až 3. místo v olympiádě přiděluj 15 bodů, za nižší místa 5.